# 7_Visualisation_using_bokeh

## Intro

This notebook uses the 'bokeh' python package to plot the EnergyPlus results.
The process is:

- loads the EnergyPlus csv file into a pandas DataFrame
- create plots

## Setup

In [15]:
import os
import pandas as pd
import bokeh.plotting as bplt
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Read csv data into a pandas DataFrame

In [16]:
fp='detached_house_0out.csv'
df=pd.read_csv(fp)
df

,Date/Time,Environment:Site Outdoor Air Drybulb Temperature [C](Hourly),DINING_ROOM:Zone Mean Air Temperature [C](Hourly),KITCHEN:Zone Mean Air Temperature [C](Hourly),LIVING_ROOM:Zone Mean Air Temperature [C](Hourly),HALL:Zone Mean Air Temperature [C](Hourly),BATHROOM:Zone Mean Air Temperature [C](Hourly),WC:Zone Mean Air Temperature [C](Hourly),BEDROOM2:Zone Mean Air Temperature [C](Hourly),BEDROOM1:Zone Mean Air Temperature [C](Hourly),BEDROOM3:Zone Mean Air Temperature [C](Hourly),LANDING:Zone Mean Air Temperature [C](Hourly),ROOF:Zone Mean Air Temperature [C](Hourly)
0,01/01 01:00:00,6.991667,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.774943
1,01/01 02:00:00,7.200000,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.743422
2,01/01 03:00:00,6.908333,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.683874
3,01/01 04:00:00,6.350000,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.600315
4,01/01 05:00:00,5.108333,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.475526
5,01/01 06:00:00,4.400000,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.358529
6,01/01 07:00:00,5.391667,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.349493
7,01/01 08:00:00,5.458333,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.327848
8,01/01 09:00:00,6.633333,21.0,21.0,21.000000,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.349165
9,01/01 10:00:00,8.441667,21.0,21.0,21.061936,21.0,21.0,21.0,21.0,21.000000,21.000000,21.0,17.499825


In [17]:
df.columns

Index(['Date/Time',
       'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
       'DINING_ROOM:Zone Mean Air Temperature [C](Hourly)',
       'KITCHEN:Zone Mean Air Temperature [C](Hourly)',
       'LIVING_ROOM:Zone Mean Air Temperature [C](Hourly)',
       'HALL:Zone Mean Air Temperature [C](Hourly)',
       'BATHROOM:Zone Mean Air Temperature [C](Hourly)',
       'WC:Zone Mean Air Temperature [C](Hourly)',
       'BEDROOM2:Zone Mean Air Temperature [C](Hourly)',
       'BEDROOM1:Zone Mean Air Temperature [C](Hourly)',
       'BEDROOM3:Zone Mean Air Temperature [C](Hourly)',
       'LANDING:Zone Mean Air Temperature [C](Hourly)',
       'ROOF:Zone Mean Air Temperature [C](Hourly) '],
      dtype='object')

### Read the Date/Time column as pandas TimeStamps

In [18]:
def parse_datetime_string(st):
    st=st.strip()
    year=2001
    month=int(st[0:2])
    day=int(st[3:5])
    hour=int(st[7:9])
    minute=int(st[10:12])
    second=(st[13:15])
    if not hour==24:
        dt=pd.Timestamp(year,month,day,hour,minute)
    else:
        hour=0
        dt=pd.Timestamp(year,month,day,hour,minute)
        dt+=pd.Timedelta('1 day')
    return dt
l=df['Date/Time']
l=[parse_datetime_string(i) for i in l]
datetime=pd.Series(data=l).values
datetime

array(['2001-01-01T01:00:00.000000000', '2001-01-01T02:00:00.000000000',
       '2001-01-01T03:00:00.000000000', ...,
       '2001-12-31T22:00:00.000000000', '2001-12-31T23:00:00.000000000',
       '2002-01-01T00:00:00.000000000'], dtype='datetime64[ns]')

## Plot the external air temperature

In [19]:
ext_temp=df['Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)'].values
ext_temp

array([ 6.99166667,  7.2       ,  6.90833333, ..., 11.26666667,
       10.6       , 10.89166667])

'ext_temp' is a pandas Series instance and can be plotted using the plot method...

In [20]:
p = bplt.figure(title='my_title',
                x_axis_label='date',
                y_axis_label='hourly external air temperature',
                plot_width=900)
p.line(x=datetime,
       y=ext_temp,
       legend='external_air_temperature',
       line_width=2)
bplt.show(p)

... this is an interactive plot...